In [1]:
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
spark = SparkSession.builder \
    .appName("Spark to MinIO") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.571") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key","minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY").getOrCreate()    

In [3]:
schema = StructType([
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("address", StringType(), True),
    StructField("phone_number", StringType(), True)
])

In [4]:
data = [
    ('Andrey', 'andrey@gmail.com', 'Rua dos Canários', '4199991111'),
    ('Joao', 'jaozinho@gmail.com', 'Rua dos Pardais', '4199992222'),
    ('Amora', 'amorinha@gmail.com', 'Rua das Andorinhas', '4199993333'),
    ('Lucy', 'lucymara@gmail.com', 'Rua dos Papagaios', '4199994444'),]


In [5]:
df = spark.createDataFrame(data, schema)

In [6]:
# Mostrar o DataFrame filtrado
df.show()

+------+------------------+------------------+------------+
|  name|             email|           address|phone_number|
+------+------------------+------------------+------------+
|Andrey|  andrey@gmail.com|  Rua dos Canários|  4199991111|
|  Joao|jaozinho@gmail.com|   Rua dos Pardais|  4199992222|
| Amora|amorinha@gmail.com|Rua das Andorinhas|  4199993333|
|  Lucy|lucymara@gmail.com| Rua dos Papagaios|  4199994444|
+------+------------------+------------------+------------+



In [7]:
df.write.mode('overwrite').parquet("s3a://bronze/teste.parquet")